In [323]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

## Load IRI data

In [324]:
drop_columns = ['Unnamed: 0',
'unit.IRI_KEY',
'unit.SY',
'unit.GE',
'unit.VEND',
'unit.ITEM',
'price.IRI_KEY',
'price.SY',
'price.GE',
'price.VEND',
'price.ITEM',
'price.cate',
'F.IRI_KEY',
'F.SY',
'F.GE',
'F.VEND',
'F.ITEM',
'F.cate',
'D.IRI_KEY',
'D.SY',
'D.GE',
'D.VEND',
'D.ITEM',
'D.cate',
'holiday.IRI_KEY',
'holiday.SY',
'holiday.GE',
'holiday.VEND',
'holiday.ITEM',
'holiday.cate',]

def read_csv(i):
    # read each slots csv, create tensor of dimension:
    # (num_timeseries, lenght, features)
    dataset = []
    data = pd.read_csv(f"dataset/iri{i}.csv", usecols=lambda x: x not in drop_columns)

    cl = data.columns.to_list()
    clcat = ['unit.cate'] 
    clu = [c for c in cl if 'unit.1' in c]
    clp = [c for c in cl if 'price.1' in c]
    clh = [c for c in cl if 'holiday.1' in c]
    clf = [c for c in cl if 'F.' in c]
    cld = [c for c in cl if 'D.' in c]

    
    dataset.append(data[clu].values)
    #replace some inf values in price
    data_clp = data[clp].replace(np.inf, np.nan).interpolate()
    dataset.append(data_clp.values)
    dataset.append(data[clh].values)
    dataset.append(data[clf].values)
    dataset.append(data[cld].values)
    dataset = np.array(dataset)
    dataset = np.transpose(dataset,(1,2,0))
    return dataset , data[clcat].values

def normalize(d):
    # normalize unit and price for NNs
    norm = torch.nn.InstanceNorm1d(2)
    dd_norm = norm(d[:,:,:2])
    d[:,:,:2] = dd_norm
    return d

def concat_slots(fist_slot, last_slot):
    # concat slots to create whole train/valid/test dataset 
    dataset = []
    catg = []
    for i in range(fist_slot, last_slot):
        tens , cat = read_csv(i)
        dataset.append(tens)
        catg.append(cat)

    catg = np.concatenate(catg,axis=0)
    dataset = np.concatenate(dataset,axis=0)
    return dataset , catg #np.array , torch.array, np.array

def mode_indx(mode):
    if mode == "train":
        (start,end) = (1,8) #(60993, 55, 5)
    if mode == "valid":
        (start,end) = (8,11)#(22951, 55, 5)
    if mode == "test":     
        (start,end) = (11,16)#(36194, 55, 5)
    return (start,end)

def get_main_data(mode):
    start, end = mode_indx(mode)
    dataset, _ = concat_slots(start, end)
    return dataset


def get_data_slot(slot):
    dataset, _ = concat_slots(slot, slot+1)
    return dataset


def get_base_forecasts(horizon, mode):
    base_forecasters = np.load(f'base_forecasters/{horizon}/_all_npy/base_{mode}.npy')
    return base_forecasters

In [414]:
import random

random.seed(0)
np.random.seed(0)

## Feature extraction

In [477]:
model = 'FFORMA1'
horizon = 7

if model == 'FFORMA2':
    use_influentional = True
elif model == 'FFORMA1':
    use_influentional = False


In [87]:
from tsfeatures import tsfeatures
from tsfeatures import acf_features, arch_stat, crossing_points, entropy, flat_spots, holt_parameters, hurst, lumpiness, nonlinearity, pacf_features, stl_features, stability, unitroot_kpss, unitroot_pp

def reshape_for_tsfeatures(Y, dim):
    """
    Reshape the timeseries for tsfeatures
    inputs:
        Y: the timeseries (numpy array) (num_series x series_length x covariates_dim)
    """
    reshaped = []
    for id in range(Y.shape[0]):
        for time in range(Y.shape[1]):
            row = np.array([id, time])
            row = np.hstack([row, Y[id, time, dim]])
            reshaped.append(row)
    reshaped = np.vstack(reshaped)
    return pd.DataFrame.from_records(reshaped, columns=['unique_id', 'ds', 'y'])


for slot in range(1, 16):
    print(f"processing slot {slot}...")
    data = get_data_slot(slot)
    for dim in range(5):
        print('exctacting features of dim ', dim, '...')
        if dim==0:
            slice = data[:, :55-horizon, :]
        else:
            slice = data[:, :, :]
        
        panel = reshape_for_tsfeatures(slice, dim)
        if dim == 0:
            features = tsfeatures(panel, freq=4, threads=16, scale=False, features=[acf_features, arch_stat, crossing_points, entropy, flat_spots, holt_parameters, hurst, lumpiness, nonlinearity, pacf_features, stl_features, stability, unitroot_kpss, unitroot_pp])
        else:
            features = tsfeatures(panel, freq=4, threads=16, scale=False, features=[crossing_points, entropy, flat_spots, hurst, lumpiness, stability])
        
        features.to_csv(f'features/slot_{slot}_h{horizon}_dim{dim}.csv', index=False)
        

processing slot 1...
exctacting features of dim  0 ...
exctacting features of dim  1 ...
exctacting features of dim  2 ...
exctacting features of dim  3 ...
exctacting features of dim  4 ...
processing slot 2...
exctacting features of dim  0 ...
exctacting features of dim  1 ...
exctacting features of dim  2 ...
exctacting features of dim  3 ...
exctacting features of dim  4 ...
processing slot 3...
exctacting features of dim  0 ...
exctacting features of dim  1 ...
exctacting features of dim  2 ...
exctacting features of dim  3 ...
exctacting features of dim  4 ...
processing slot 4...
exctacting features of dim  0 ...
exctacting features of dim  1 ...
exctacting features of dim  2 ...
exctacting features of dim  3 ...
exctacting features of dim  4 ...
processing slot 5...
exctacting features of dim  0 ...
exctacting features of dim  1 ...
exctacting features of dim  2 ...
exctacting features of dim  3 ...
exctacting features of dim  4 ...
processing slot 6...
exctacting features of d

In [478]:
train_main_data = get_main_data("train")
valid_main_data = get_main_data("valid")
test_main_data = get_main_data("test")

In [479]:
train_base_data = get_base_forecasts(horizon, "train")
valid_base_data = get_base_forecasts(horizon, "valid")
test_base_data = get_base_forecasts(horizon, "test")

In [480]:
def loss_fn(y_true, y_pred, loss):
    """
    Loss function for the meta-learner
    Args:
        y_true: true values (numpy array) (num_series x horizon)
        Y_pred: the matrix of predictions (num_series x num_models x horizon)
    """
    if loss == 'mse':
        return np.mean(np.square(y_true - y_pred), axis=2)
    elif loss == 'rmse':
        return np.sqrt(np.mean(np.square(y_true - y_pred), axis=2))
    elif loss == 'mae':
        return np.mean(np.abs(y_true - y_pred), axis=2)
    elif callable(loss):
        return loss(y_true, y_pred)
    else:
        raise ValueError('Loss function not supported')

In [481]:
def labels_selection(Y_pred, y_true, loss, return_one_hot=False):
    """
    Generate labels for the meta-learner using selection
    Args:
        y_true: true values (numpy array) (num_series x horizon)
        Y_pred: the matrix of predictions (num_series x num_models x horizon)
    """
    error = loss_fn(y_true, np.transpose(Y_pred, (1, 0, 2)), loss)
    error = error.T
    label_indices = np.argmin(error, axis=1)
    if return_one_hot:    
        # convert to one-hot encoding
        labels = np.zeros(Y_pred.shape[1:])
        for i in range(Y_pred.shape[1]):
            labels[i, label_indices[i]] = 1.0
        return labels
    else:
        return label_indices

In [482]:
train_Y = labels_selection(train_base_data, train_main_data[:, -horizon:, 0], 'rmse')
valid_Y = labels_selection(valid_base_data, valid_main_data[:, -horizon:, 0], 'rmse')
test_Y = labels_selection(test_base_data, test_main_data[:, -horizon:, 0], 'rmse')

In [483]:
def load_features(mode):
    start, end = mode_indx(mode)
    # read csv files
    slots = []
    if use_influentional:
        dim_range = 5
    else:
        dim_range = 1
    for slot in range(start, end):
        slot_dims = []
        for dim in range(dim_range):
            slot_dim_df = pd.read_csv(f'features/slot_{slot}_h{horizon}_dim{dim}.csv')
            slot_dims.append(slot_dim_df.add_prefix(f'dim_{dim}_'))
        slots.append(pd.concat(slot_dims, axis=1))
    combined = pd.concat(slots).reset_index().drop(['index'] + [f'dim_{d}_unique_id' for d in range(dim_range)], axis=1).fillna(0)
    return combined

In [484]:
train_X = load_features("train").values
valid_X = load_features("valid").values
test_X = load_features("test").values

## Selecting best forecaster using XGboost

In [485]:
import xgboost as xgb

In [486]:
bst = xgb.XGBClassifier(eta=0.575188, max_depth=14, subsample=0.9161483,
                        colsample_bytree = 0.7670739, objective='multi:softmax', num_class=8,
                        random_state=0, n_estimators=100, verbosity=2, tree_method='gpu_hist', gpu_id=0,
                        early_stopping_rounds=1)

In [487]:
bst.fit(train_X, train_Y, eval_set=[(valid_X, valid_Y)])

[0]	validation_0-mlogloss:2.11179


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7670739, early_stopping_rounds=1,
              enable_categorical=False, eta=0.575188, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=14,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_class=8, num_parallel_tree=None, ...)

In [488]:
def generate_forecasts(labels, base_data):
    forecasts = []
    for i in range(base_data.shape[0]):
        forecasts.append(base_data[i, labels[i], :])
    return np.vstack(forecasts)

In [489]:
train_predictions = generate_forecasts(bst.predict(train_X), train_base_data)
valid_predictions = generate_forecasts(bst.predict(valid_X), valid_base_data)
test_predictions = generate_forecasts(bst.predict(test_X), test_base_data)
combined_pred = np.vstack((train_predictions, valid_predictions, test_predictions))

In [490]:
pd.DataFrame.from_records(combined_pred).to_csv(f'{model}-h{horizon}', index=False)